<a href="https://colab.research.google.com/github/boyu571/kpc_lecture_notes/blob/master/(%EC%9E%90%EB%8F%99%ED%99%94)time%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
########  Step.01  ########
import numpy as np
import pandas as pd
from google.colab import files
files.upload()

{}

In [ ]:
########  Step.02  ########
#파일 종류 설정 (read_excel or read_csv)
df = pd.read_excel('time분석데이터.xlsx')
df

,구분,단위,월평균 물동량,월평균 도급비(백만원/월),현근무인원(인/日),작업일수(일),근무시간(시간/日),Standard_Time(EC/개)
0,검수,PCS,1893233,44.583456,16.0,30,8,4.100246
1,입고/적치,box,62522,14.007040,6.0,30,8,41.774362
2,피킹,box,56856,30.484800,15.0,30,8,121.236721
3,유통가공1,PCS,1705680,27.524960,10.0,30,8,3.500000
4,유통가공2,PCS,511704,44.869368,15.0,30,8,9.100000
5,유통가공3,PCS,341136,49.320832,23.0,30,8,28.064056
6,유통가공4,PCS,852840,61.578320,22.0,30,8,10.103060
7,출고 대기,box,56856,5.000000,2.5,30,8,15.677162
8,출고,box,56856,22.785350,10.0,30,8,75.772951


# **실제 분석**
#### 물동량증감율=1
#### 순작업시간비율=0.75

In [ ]:
########  Step.03  ########
물동량증감율=1
순작업시간비율=0.75

########  Step.04  ########
df['순작업시간(시간/日)'] = df['근무시간(시간/日)']*순작업시간비율
df['일작업_목표량(물량/일)'] = df['월평균 물동량']/df['작업일수(일)']*물동량증감율
df['TaktTime(SEC/개)'] = (df['순작업시간(시간/日)'] /df['일작업_목표량(물량/일)'])*3600
df['적정인력(명)'] = df['Standard_Time(EC/개)'] /df['TaktTime(SEC/개)']

df['도급비(백만원/월)'] = df['월평균 도급비(백만원/월)'] /df['현근무인원(인/日)']*df['적정인력(명)']

df['인력가감(명)'] =  (df['적정인력(명)'] - df['현근무인원(인/日)']).apply(lambda x: round(x))
df['도급비_증감액(백만원/월)'] = (df['월평균 도급비(백만원/월)'] /df['현근무인원(인/日)']*df['인력가감(명)']).apply(lambda x: round(x))
df

,구분,단위,월평균 물동량,월평균 도급비(백만원/월),현근무인원(인/日),작업일수(일),근무시간(시간/日),Standard_Time(EC/개),순작업시간(시간/日),일작업_목표량(물량/일),TaktTime(SEC/개),적정인력(명),도급비(백만원/월),인력가감(명),도급비_증감액(백만원/월)
0,검수,PCS,1893233,44.583456,16.0,30,8,4.100246,6.0,63107.766667,0.342272,11.979508,33.380492,-4,-11
1,입고/적치,box,62522,14.007040,6.0,30,8,41.774362,6.0,2084.066667,10.364352,4.030581,9.409419,-2,-5
2,피킹,box,56856,30.484800,15.0,30,8,121.236721,6.0,1895.200000,11.397214,10.637400,21.618600,-4,-8
3,유통가공1,PCS,1705680,27.524960,10.0,30,8,3.500000,6.0,56856.000000,0.379907,9.212778,25.358134,-1,-3
4,유통가공2,PCS,511704,44.869368,15.0,30,8,9.100000,6.0,17056.800000,1.266357,7.185967,21.495319,-8,-24
5,유통가공3,PCS,341136,49.320832,23.0,30,8,28.064056,6.0,11371.200000,1.899536,14.774166,31.681486,-8,-17
6,유통가공4,PCS,852840,61.578320,22.0,30,8,10.103060,6.0,28428.000000,0.759814,13.296750,37.217796,-9,-25
7,출고 대기,box,56856,5.000000,2.5,30,8,15.677162,6.0,1895.200000,11.397214,1.375526,2.751052,-1,-2
8,출고,box,56856,22.785350,10.0,30,8,75.772951,6.0,1895.200000,11.397214,6.648375,15.148555,-3,-7


In [ ]:
########  Step.05  ########
#원하는 결과값 뽑아보기
# 도급비(백만원/월) 컬럼의 합계 계산
total_도급비 = df['월평균 도급비(백만원/월)'].sum()
total_현재근무인원 = df['현근무인원(인/日)'].sum()
total_TaktTime = df['TaktTime(SEC/개)'].sum()
total_Standard_Time = df['Standard_Time(EC/개)'].sum()
total_적정인력_도급비 = df['도급비(백만원/월)'].sum()
total_적정인력 = df['적정인력(명)'].sum()

total_인력가감 = df['인력가감(명)'].sum()
total_도급비_증감액 = df['도급비_증감액(백만원/월)'].sum()
# 변수와 합계 값으로 데이터프레임 생성
data = {
    '변수': ['월평균 도급비(백만원/월)', '현근무인원(인/日)', 'TaktTime(SEC/개)', 'Standard_Time(EC/개)', 'total_적정인력', '도급비(백만원/월)', '인력가감(명)', '도급비_증감액(백만원/월)'],
    '합계': [total_도급비, total_현재근무인원, total_TaktTime, total_Standard_Time, total_적정인력, total_적정인력_도급비, total_인력가감, total_도급비_증감액]
}

df_total = pd.DataFrame(data)
df_total

,변수,합계
0,월평균 도급비(백만원/월),300.154126
1,현근무인원(인/日),119.500000
2,TaktTime(SEC/개),49.203880
3,Standard_Time(EC/개),309.328558
4,total_적정인력,79.141050
5,도급비(백만원/월),198.060852
6,인력가감(명),-40.000000
7,도급비_증감액(백만원/월),-102.000000


# **AS - IS** 조건을 변경하여 시뮬레이션을 해본다.

#### 물동량증감율=1.2
#### 순작업시간비율=0.75

##### 순작업시간비율을 KPI로 가져가며 TARGET을 정한다.& 작업일수도 5일 줄여준다.

In [ ]:
########  Step.06  ########
물동량증감율=1.2
순작업시간비율=0.75
# 5일을 줄일 수 있다고 가정
df['작업일수(일)'] = df['작업일수(일)']-5
########  Step.07  ########
df['순작업시간(시간/日)'] = df['근무시간(시간/日)']*순작업시간비율
df['일작업_목표량(물량/일)'] = df['월평균 물동량']/df['작업일수(일)']*물동량증감율
df['TaktTime(SEC/개)'] = (df['순작업시간(시간/日)'] /df['일작업_목표량(물량/일)'])*3600
df['적정인력(명)'] = df['Standard_Time(EC/개)'] /df['TaktTime(SEC/개)']

df['도급비(백만원/월)'] = df['월평균 도급비(백만원/월)'] /df['현근무인원(인/日)']*df['적정인력(명)']

df['인력가감(명)'] =  (df['적정인력(명)'] - df['현근무인원(인/日)']).apply(lambda x: round(x))
df['도급비_증감액(백만원/월)'] = (df['월평균 도급비(백만원/월)'] /df['현근무인원(인/日)']*df['인력가감(명)']).apply(lambda x: round(x))

In [ ]:
########  Step.08  ########
#원하는 결과값 뽑아보기
# 도급비(백만원/월) 컬럼의 합계 계산
total_도급비 = df['월평균 도급비(백만원/월)'].sum()
total_현재근무인원 = df['현근무인원(인/日)'].sum()
total_TaktTime = df['TaktTime(SEC/개)'].sum()
total_Standard_Time = df['Standard_Time(EC/개)'].sum()
total_적정인력 = df['적정인력(명)'].sum()

total_적정인력_도급비 = df['도급비(백만원/월)'].sum()
total_인력가감 = df['인력가감(명)'].sum()
total_도급비_증감액 = df['도급비_증감액(백만원/월)'].sum()
# 변수와 합계 값으로 데이터프레임 생성
data = {
    '변수': ['월평균 도급비(백만원/월)', '현근무인원(인/日)', 'TaktTime(SEC/개)', 'Standard_Time(EC/개)', 'total_적정인력', '도급비(백만원/월)', '인력가감(명)', '도급비_증감액(백만원/월)'],
    '합계': [total_도급비, total_현재근무인원, total_TaktTime, total_Standard_Time, total_적정인력, total_적정인력_도급비, total_인력가감, total_도급비_증감액]
}

df_total = pd.DataFrame(data)
df_total

,변수,합계
0,월평균 도급비(백만원/월),300.154126
1,현근무인원(인/日),119.500000
2,TaktTime(SEC/개),27.335489
3,Standard_Time(EC/개),309.328558
4,total_적정인력,142.453890
5,도급비(백만원/월),356.509533
6,인력가감(명),24.000000
7,도급비_증감액(백만원/월),60.000000


In [ ]:
asis = df_total.transpose()
asis.columns = asis.iloc[0] #1행을 컬럼으로 사용
asis = asis[1:].reset_index(drop=True) #1행을 컬럼으로 사용
"""DataFrame에서 첫 번째 행을 제거하고 인덱스를 재설정하는 작업입니다.
asis[1:]: 첫 번째 행을 제외한 나머지 행을 선택합니다.
reset_index(drop=True): 인덱스를 재설정하고 이전 인덱스를 제거합니다. drop=True 옵션을 사용하여 이전 인덱스를 제거합니다.
결과적으로 asis DataFrame은 첫 번째 행이 제거되고 인덱스가 재설정된 상태로 업데이트됩니다."""
asis

변수,월평균 도급비(백만원/월),현근무인원(인/日),TaktTime(SEC/개),Standard_Time(EC/개),total_적정인력,도급비(백만원/월),인력가감(명),도급비_증감액(백만원/월)
0,300.154126,119.5,27.335489,309.328558,142.45389,356.509533,24.0,60.0


In [ ]:
asis[['total_적정인력', '도급비(백만원/월)']]

변수,total_적정인력,도급비(백만원/월)
0,142.45389,356.509533


In [ ]:
########  Step.09  ########
import numpy as np
import pandas as pd
from google.colab import files
files.upload()

In [ ]:
########  Step.10  ########
#파일 종류 설정 (read_excel or read_csv)
df1 = pd.read_excel('time분석데이터_자동화조건.xlsx', sheet_name='표준시간 절감율')
df2 = pd.read_excel('time분석데이터_자동화조건.xlsx', sheet_name='투입금액')

In [ ]:
########  Step.11  ####### #  투자 금액 입력
a1= 4200
a2= 8000

########  Step.12  ########
df1['순작업시간(시간/日)'] = df1['근무시간(시간/日)']*순작업시간비율
df1['일작업_목표량(물량/일)'] = df1['월평균 물동량']/df1['작업일수(일)']*물동량증감율
df1['TaktTime(SEC/개)'] = (df1['순작업시간(시간/日)'] /df1['일작업_목표량(물량/일)'])*3600
#As_is_StandardTime = Standard_Time(EC/개)
df1


,구분,단위,월평균 물동량,월평균 도급비(백만원/월),현근무인원(인/日),작업일수(일),근무시간(시간/日),Standard_Time(EC/개),순작업시간(시간/日),일작업_목표량(물량/일),TaktTime(SEC/개)
0,검수,PCS,1893233,44.583456,16.0,25,8,4.100246,6.0,151458.64,0.142613
1,입고/적치,box,62522,14.007040,6.0,25,8,41.774362,6.0,5001.76,4.318480
2,피킹,box,56856,30.484800,15.0,25,8,121.236721,6.0,4548.48,4.748839
3,유통 1,PCS,1705680,27.524960,10.0,25,8,3.500000,6.0,136454.40,0.158295
4,유통 2,PCS,511704,44.869368,15.0,25,8,9.100000,6.0,40936.32,0.527649
5,유통 3,PCS,341136,49.320832,23.0,25,8,28.064056,6.0,27290.88,0.791473
6,유통 4,PCS,852840,61.578320,22.0,25,8,10.103060,6.0,68227.20,0.316589
7,출고 대기,box,56856,5.000000,2.5,25,8,15.677162,6.0,4548.48,4.748839
8,출고,box,56856,22.785350,10.0,25,8,75.772951,6.0,4548.48,4.748839


In [ ]:
########  Step.13  ######## 데이터 합치기
merged_df = pd.merge(df1, df2, on='구분')
merged_df

,구분,단위,월평균 물동량,월평균 도급비(백만원/월),현근무인원(인/日),작업일수(일),근무시간(시간/日),Standard_Time(EC/개),순작업시간(시간/日),일작업_목표량(물량/일),TaktTime(SEC/개),자동화,As-is,자동화 1안,자동화 2안
0,검수,PCS,1893233,44.583456,16.0,25,8,4.100246,6.0,90875.184,0.237689,자동 검수,0,0.20,0.40
1,입고/적치,box,62522,14.007040,6.0,25,8,41.774362,6.0,3001.056,7.197466,자동 컨배이어 시스템,0,0.15,0.15
2,피킹,box,56856,30.484800,15.0,25,8,121.236721,6.0,2729.088,7.914732,자동 피킹 시스템,0,0.20,0.30
3,유통 1,PCS,1705680,27.524960,10.0,25,8,3.500000,6.0,81872.640,0.263824,자동 포장,0,0.40,0.55
4,유통 2,PCS,511704,44.869368,15.0,25,8,9.100000,6.0,24561.792,0.879415,작업대 Levler,0,0.30,0.40
5,유통 3,PCS,341136,49.320832,23.0,25,8,28.064056,6.0,16374.528,1.319122,포장 기계 자동화,0,0.40,0.60
6,유통 4,PCS,852840,61.578320,22.0,25,8,10.103060,6.0,40936.320,0.527649,벤딩 머신,0,0.20,0.34
7,출고 대기,box,56856,5.000000,2.5,25,8,15.677162,6.0,2729.088,7.914732,대기 공간 이적 자동화,0,0.28,0.43
8,출고,box,56856,22.785350,10.0,25,8,75.772951,6.0,2729.088,7.914732,상차 컨베이어,0,0.18,0.32


# **1안 계산**

In [ ]:
########  Step.14  ########
merged_df['as_is(Standard Time)'] = merged_df['Standard_Time(EC/개)']
merged_df['to_be(Standard Time)'] = merged_df['Standard_Time(EC/개)']*(1-merged_df['자동화 1안'])
merged_df['적정인력(명)'] = merged_df['to_be(Standard Time)'] / merged_df['TaktTime(SEC/개)']
merged_df['도급비(백만원/월)'] = merged_df['월평균 도급비(백만원/월)'] / merged_df['현근무인원(인/日)'] * merged_df['적정인력(명)']

merged_df

,구분,단위,월평균 물동량,월평균 도급비(백만원/월),현근무인원(인/日),작업일수(일),근무시간(시간/日),Standard_Time(EC/개),순작업시간(시간/日),일작업_목표량(물량/일),TaktTime(SEC/개),자동화,As-is,자동화 1안,자동화 2안,as_is(Standard Time),to_be(Standard Time),적정인력(명),도급비(백만원/월)
0,검수,PCS,1893233,44.583456,16.0,25,8,4.100246,6.0,90875.184,0.237689,자동 검수,0,0.20,0.40,4.100246,3.280197,13.800393,38.454327
1,입고/적치,box,62522,14.007040,6.0,25,8,41.774362,6.0,3001.056,7.197466,자동 컨배이어 시스템,0,0.15,0.15,41.774362,35.508207,4.933431,11.517129
2,피킹,box,56856,30.484800,15.0,25,8,121.236721,6.0,2729.088,7.914732,자동 피킹 시스템,0,0.20,0.30,121.236721,96.989377,12.254285,24.904627
3,유통 1,PCS,1705680,27.524960,10.0,25,8,3.500000,6.0,81872.640,0.263824,자동 포장,0,0.40,0.55,3.500000,2.100000,7.959840,21.909428
4,유통 2,PCS,511704,44.869368,15.0,25,8,9.100000,6.0,24561.792,0.879415,작업대 Levler,0,0.30,0.40,9.100000,6.370000,7.243454,21.667281
5,유통 3,PCS,341136,49.320832,23.0,25,8,28.064056,6.0,16374.528,1.319122,포장 기계 자동화,0,0.40,0.60,28.064056,16.838434,12.764880,27.372804
6,유통 4,PCS,852840,61.578320,22.0,25,8,10.103060,6.0,40936.320,0.527649,벤딩 머신,0,0.20,0.34,10.103060,8.082448,15.317856,42.874901
7,출고 대기,box,56856,5.000000,2.5,25,8,15.677162,6.0,2729.088,7.914732,대기 공간 이적 자동화,0,0.28,0.43,15.677162,11.287557,1.426145,2.852290
8,출고,box,56856,22.785350,10.0,25,8,75.772951,6.0,2729.088,7.914732,상차 컨베이어,0,0.18,0.32,75.772951,62.133820,7.850401,17.887413


In [ ]:
merged_df.columns

Index(['구분', '단위', '월평균 물동량', '월평균 도급비(백만원/월)', '현근무인뤈(인/日)', '작업일수(일)',
       '근무시간(시간/日)', 'Standard_Time(EC/개)', '순작업시간(시간/日)', '일작업_목표량(물량/일)',
       'TaktTime(SEC/개)', '기계화 자동화 도입', ' 표준시간 절감율_자동화 1안', ' 표준시간 절감율_자동화 2안',
       'as_is(Standard Time)', 'to_be(Standard Time)', '적정인력(명)',
       '도급비(백만원/월)'],
      dtype='object')

In [ ]:
########  Step.15  ########

# 컬럼들의 합계 계산
sum_columns = ['월평균 도급비(백만원/월)', '현근무인원(인/日)',  'as_is(Standard Time)', 'to_be(Standard Time)', '적정인력(명)', '도급비(백만원/월)']
column_sums = merged_df[sum_columns].sum()

# 합계를 데이터프레임으로 변환
sum_df1 = pd.DataFrame(column_sums, columns=['1안(4200만원)'])
sum_df1

,1안(4200만원)
월평균 도급비(백만원/월),300.154126
현근무인원(인/日),119.500000
as_is(Standard Time),309.328558
to_be(Standard Time),242.590040
적정인력(명),83.550685
도급비(백만원/월),209.440200


# **2안 계산**

In [ ]:
########  Step.16  ########
merged_df['as_is'] = merged_df['Standard_Time(EC/개)']
merged_df['to_be'] = merged_df['Standard_Time(EC/개)']*(1-merged_df['자동화 2안'])
merged_df['적정인력(명)'] = merged_df['to_be'] / merged_df['TaktTime(SEC/개)']
merged_df['도급비(백만원/월)'] = merged_df['월평균 도급비(백만원/월)'] / merged_df['현근무인원(인/日)'] * merged_df['적정인력(명)']


########  Step.17  ########
# 컬럼들의 합계 계산
sum_columns = ['월평균 도급비(백만원/월)', '현근무인원(인/日)',  'as_is(Standard Time)', 'to_be(Standard Time)', '적정인력(명)', '도급비(백만원/월)']
column_sums = merged_df[sum_columns].sum()

# 합계를 데이터프레임으로 변환
sum_df2 = pd.DataFrame(column_sums, columns=['2안(8000만원)'])

# 결과 출력
sum_df2

,2안(8000만원)
월평균 도급비(백만원/월),300.154126
현근무인원(인/日),119.500000
as_is(Standard Time),309.328558
to_be(Standard Time),242.590040
적정인력(명),66.971051
도급비(백만원/월),167.865306


## **투자효과 다이나믹 시뮬레이션**

In [ ]:
########  Step.18  ########
# sum_df1과 sum_df2를 옆으로 붙이기
combined_df = pd.concat([sum_df1, sum_df2], axis=1)
combined_df=combined_df.transpose()
combined_df

,월평균 도급비(백만원/월),현근무인원(인/日),as_is(Standard Time),to_be(Standard Time),적정인력(명),도급비(백만원/월)
1안(4200만원),300.154126,119.5,309.328558,242.59004,83.550685,209.440200
2안(8000만원),300.154126,119.5,309.328558,242.59004,66.971051,167.865306


In [ ]:
asis[['total_적정인력', '도급비(백만원/월)']]

변수,total_적정인력,도급비(백만원/월)
0,142.45389,356.509533


In [ ]:
combined_df['As-is_적정인력'] = asis['total_적정인력'].values[0]
combined_df['As-is_월도급비'] = asis['도급비(백만원/월)'].values[0]
combined_df['As-is_연간도급비'] = combined_df['As-is_월도급비'] *12
combined_df['연간도급비(백만원/연)'] = combined_df['도급비(백만원/월)'] *12
combined_df.columns

Index(['월평균 도급비(백만원/월)', '현근무인원(인/日)', 'as_is(Standard Time)',
       'to_be(Standard Time)', '적정인력(명)', '도급비(백만원/월)', 'As-is_적정인력',
       'As-is_월도급비', 'As-is_연간도급비', '연간도급비(백만원/연)'],
      dtype='object')

In [1]:
df_last= combined_df[[ '적정인력(명)', '연간도급비(백만원/연)', 'As-is_적정인력',
       'As-is_월도급비', 'As-is_연간도급비']].copy()
df_last

NameError: name 'combined_df' is not defined

In [ ]:
df_last['도급인력 절감'] = df_last['As-is_적정인력'] - df_last['적정인력(명)']
df_last['연간 도급비 절감(백만원/연)'] = df_last['As-is_연간도급비'] - df_last['연간도급비(백만원/연)']
df_last['투자금액'] = [4200,8000]
df_last['투자회수 기간(년차)'] = df_last['투자금액'] / df_last['연간 도급비 절감(백만원/연)']
round(df_last,1)

<ipython-input-86-334e68f98098>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_last['도급인력 절감'] = df_last['As-is_적정인력'] - df_last['적정인력(명)']
<ipython-input-86-334e68f98098>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_last['연간 도급비 절감(백만원/연)'] = df_last['As-is_연간도급비'] - df_last['연간도급비(백만원/연)']
<ipython-input-86-334e68f98098>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

,적정인력(명),연간도급비(백만원/연),As-is_적정인력,As-is_월도급비,As-is_연간도급비,도급인력 절감,연간 도급비 절감(백만원/연),투자금액,투자회수 기간(년차)
1안(4200만원),83.6,2513.3,142.5,356.5,4278.1,58.9,1764.8,4200,2.4
2안(8000만원),67.0,2014.4,142.5,356.5,4278.1,75.5,2263.7,8000,3.5


In [3]:
df1 = pd.read_excel('time분석데이터_자동화조건.xlsx', sheet_name='표준시간 절감율')
df2 = pd.read_excel('time분석데이터_자동화조건.xlsx', sheet_name='투입금액')

In [4]:
물동량증감율_list = np.arange(0.8, 2.1, 0.1)
results = []

for 물동량증감율 in 물동량증감율_list:
    순작업시간비율 = 0.75

    ########  Step.07  ########
    df1['순작업시간(시간/日)'] = df1['근무시간(시간/日)']*순작업시간비율
    df1['일작업_목표량(물량/일)'] = df1['월평균 물동량']/df1['작업일수(일)']*물동량증감율
    df1['TaktTime(SEC/개)'] = (df1['순작업시간(시간/日)'] /df1['일작업_목표량(물량/일)'])*3600
    df1['적정인력(명)'] = df1['Standard_Time(EC/개)'] /df1['TaktTime(SEC/개)']

    df1['도급비(백만원/월)'] = df1['월평균 도급비(백만원/월)'] /df1['현근무인원(인/日)']*df1['적정인력(명)']

    df1['인력가감(명)'] =  (df1['적정인력(명)'] - df1['현근무인원(인/日)']).apply(lambda x: round(x))
    df1['도급비_증감액(백만원/월)'] = (df1['월평균 도급비(백만원/월)'] /df1['현근무인원(인/日)']*df1['인력가감(명)']).apply(lambda x: round(x))

    ########  Step.08  ########
    #원하는 결과값 뽑아보기
    # 도급비(백만원/월) 컬럼의 합계 계산
    total_도급비 = df1['월평균 도급비(백만원/월)'].sum()
    total_현재근무인원 = df1['현근무인원(인/日)'].sum()
    total_TaktTime = df1['TaktTime(SEC/개)'].sum()
    total_Standard_Time = df1['Standard_Time(EC/개)'].sum()
    total_적정인력 = df1['적정인력(명)'].sum()

    total_적정인력_도급비 = df1['도급비(백만원/월)'].sum()
    total_인력가감 = df1['인력가감(명)'].sum()
    total_도급비_증감액 = df1['도급비_증감액(백만원/월)'].sum()
    # 변수와 합계 값으로 데이터프레임 생성
    data = {
        '변수': ['월평균 도급비(백만원/월)', '현근무인원(인/日)', 'TaktTime(SEC/개)', 'Standard_Time(EC/개)', 'total_적정인력', '도급비(백만원/월)', '인력가감(명)', '도급비_증감액(백만원/월)'],
        '합계': [total_도급비, total_현재근무인원, total_TaktTime, total_Standard_Time, total_적정인력, total_적정인력_도급비, total_인력가감, total_도급비_증감액]
    }

    df_total = pd.DataFrame(data)
    asis = df_total.transpose()
    asis.columns = asis.iloc[0] #1행을 컬럼으로 사용
    asis = asis[1:].reset_index(drop=True) #1행을 컬럼으로 사용

    ########  Step.11  ####### #  투자 금액 입력
    a1= 4200
    a2= 8000

    ########  Step.12  ########
    df1['순작업시간(시간/日)'] = df1['근무시간(시간/日)']*순작업시간비율
    df1['일작업_목표량(물량/일)'] = df1['월평균 물동량']/df1['작업일수(일)']*물동량증감율
    df1['TaktTime(SEC/개)'] = (df1['순작업시간(시간/日)'] /df1['일작업_목표량(물량/일)'])*3600
    #As_is_StandardTime = Standard_Time(EC/개)


    ########  Step.13  ######## 데이터 합치기
    merged_df = pd.merge(df1, df2, on='구분')
    ########  Step.14  ########
    merged_df['as_is(Standard Time)'] = merged_df['Standard_Time(EC/개)']
    merged_df['to_be(Standard Time)'] = merged_df['Standard_Time(EC/개)']*(1-merged_df['자동화 1안'])
    merged_df['적정인력(명)'] = merged_df['to_be(Standard Time)'] / merged_df['TaktTime(SEC/개)']
    merged_df['도급비(백만원/월)'] = merged_df['월평균 도급비(백만원/월)'] / merged_df['현근무인원(인/日)'] * merged_df['적정인력(명)']



    ########  Step.15  ########

    # 컬럼들의 합계 계산
    sum_columns = ['월평균 도급비(백만원/월)', '현근무인원(인/日)',  'as_is(Standard Time)', 'to_be(Standard Time)', '적정인력(명)', '도급비(백만원/월)']
    column_sums = merged_df[sum_columns].sum()

    # 합계를 데이터프레임으로 변환
    sum_df1 = pd.DataFrame(column_sums, columns=['1안(4200백만원)'])
    ########  Step.16  ########
    merged_df['as_is'] = merged_df['Standard_Time(EC/개)']
    merged_df['to_be'] = merged_df['Standard_Time(EC/개)']*(1-merged_df['자동화 2안'])
    merged_df['적정인력(명)'] = merged_df['to_be'] / merged_df['TaktTime(SEC/개)']
    merged_df['도급비(백만원/월)'] = merged_df['월평균 도급비(백만원/월)'] / merged_df['현근무인원(인/日)'] * merged_df['적정인력(명)']


    ########  Step.17  ########
    # 컬럼들의 합계 계산
    sum_columns = ['월평균 도급비(백만원/월)', '현근무인원(인/日)',  'as_is(Standard Time)', 'to_be(Standard Time)', '적정인력(명)', '도급비(백만원/월)']
    column_sums = merged_df[sum_columns].sum()

    # 합계를 데이터프레임으로 변환
    sum_df2 = pd.DataFrame(column_sums, columns=['2안(8000백만원)'])
    ########  Step.18  ########
    # sum_df1과 sum_df2를 옆으로 붙이기
    combined_df = pd.concat([sum_df1, sum_df2], axis=1)
    combined_df=combined_df.transpose()
    asis[['total_적정인력', '도급비(백만원/월)']]

    combined_df['As-is_적정인력'] = asis['total_적정인력'].values[0]
    combined_df['As-is_월도급비'] = asis['도급비(백만원/월)'].values[0]
    combined_df['As-is_연간도급비'] = combined_df['As-is_월도급비'] *12
    combined_df['연간도급비(백만원/연)'] = combined_df['도급비(백만원/월)'] *12
    df_last= combined_df[[ '적정인력(명)', '연간도급비(백만원/연)', 'As-is_적정인력',
        'As-is_월도급비', 'As-is_연간도급비']].copy()

    df_last['도급인력 절감'] = df_last['As-is_적정인력'] - df_last['적정인력(명)']
    df_last['연간 도급비 절감(백만원/연)'] = df_last['As-is_연간도급비'] - df_last['연간도급비(백만원/연)']
    df_last['투자금액'] = [4200,8000]
    df_last['투자회수 기간(년차)'] = df_last['투자금액'] / df_last['연간 도급비 절감(백만원/연)']
    q1 = round(df_last,1)
    q1 = q1[['투자회수 기간(년차)']]
    q1['물동량증감율'] = f'{물동량증감율:.1f}'
    #q1.set_index('비중', inplace=True)
    q1 = q1.transpose()

    sum_df = pd.DataFrame(q1)
    results.append(sum_df)

# 결과 데이터프레임 생성
combined_df = pd.concat(results, axis=1)
combined_df.transpose()

<ipython-input-4-391e86428e70>:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_last['도급인력 절감'] = df_last['As-is_적정인력'] - df_last['적정인력(명)']
<ipython-input-4-391e86428e70>:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_last['연간 도급비 절감(백만원/연)'] = df_last['As-is_연간도급비'] - df_last['연간도급비(백만원/연)']
<ipython-input-4-391e86428e70>:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

,투자회수 기간(년차),비중
1안(4200백만원),6.9,0.8
2안(8000백만원),8.5,0.8
1안(4200백만원),6.2,0.9
2안(8000백만원),7.6,0.9
1안(4200백만원),5.5,1.0
2안(8000백만원),6.8,1.0
1안(4200백만원),5.0,1.1
2안(8000백만원),6.2,1.1
1안(4200백만원),4.6,1.2
2안(8000백만원),5.7,1.2


In [10]:
순작업시간비율_list = np.arange(0.4, 0.9, 0.05)
results = []

for 순작업시간비율 in 순작업시간비율_list:
    물동량증감율 = 1.2

    ########  Step.07  ########
    df1['순작업시간(시간/日)'] = df1['근무시간(시간/日)']*순작업시간비율
    df1['일작업_목표량(물량/일)'] = df1['월평균 물동량']/df1['작업일수(일)']*물동량증감율
    df1['TaktTime(SEC/개)'] = (df1['순작업시간(시간/日)'] /df1['일작업_목표량(물량/일)'])*3600
    df1['적정인력(명)'] = df1['Standard_Time(EC/개)'] /df1['TaktTime(SEC/개)']

    df1['도급비(백만원/월)'] = df1['월평균 도급비(백만원/월)'] /df1['현근무인원(인/日)']*df1['적정인력(명)']

    df1['인력가감(명)'] =  (df1['적정인력(명)'] - df1['현근무인원(인/日)']).apply(lambda x: round(x))
    df1['도급비_증감액(백만원/월)'] = (df1['월평균 도급비(백만원/월)'] /df1['현근무인원(인/日)']*df1['인력가감(명)']).apply(lambda x: round(x))

    ########  Step.08  ########
    #원하는 결과값 뽑아보기
    # 도급비(백만원/월) 컬럼의 합계 계산
    total_도급비 = df1['월평균 도급비(백만원/월)'].sum()
    total_현재근무인원 = df1['현근무인원(인/日)'].sum()
    total_TaktTime = df1['TaktTime(SEC/개)'].sum()
    total_Standard_Time = df1['Standard_Time(EC/개)'].sum()
    total_적정인력 = df1['적정인력(명)'].sum()

    total_적정인력_도급비 = df1['도급비(백만원/월)'].sum()
    total_인력가감 = df1['인력가감(명)'].sum()
    total_도급비_증감액 = df1['도급비_증감액(백만원/월)'].sum()
    # 변수와 합계 값으로 데이터프레임 생성
    data = {
        '변수': ['월평균 도급비(백만원/월)', '현근무인원(인/日)', 'TaktTime(SEC/개)', 'Standard_Time(EC/개)', 'total_적정인력', '도급비(백만원/월)', '인력가감(명)', '도급비_증감액(백만원/월)'],
        '합계': [total_도급비, total_현재근무인원, total_TaktTime, total_Standard_Time, total_적정인력, total_적정인력_도급비, total_인력가감, total_도급비_증감액]
    }

    df_total = pd.DataFrame(data)
    asis = df_total.transpose()
    asis.columns = asis.iloc[0] #1행을 컬럼으로 사용
    asis = asis[1:].reset_index(drop=True) #1행을 컬럼으로 사용

    ########  Step.11  ####### #  투자 금액 입력
    a1= 4200
    a2= 8000

    ########  Step.12  ########
    df1['순작업시간(시간/日)'] = df1['근무시간(시간/日)']*순작업시간비율
    df1['일작업_목표량(물량/일)'] = df1['월평균 물동량']/df1['작업일수(일)']*물동량증감율
    df1['TaktTime(SEC/개)'] = (df1['순작업시간(시간/日)'] /df1['일작업_목표량(물량/일)'])*3600
    #As_is_StandardTime = Standard_Time(EC/개)


    ########  Step.13  ######## 데이터 합치기
    merged_df = pd.merge(df1, df2, on='구분')
    ########  Step.14  ########
    merged_df['as_is(Standard Time)'] = merged_df['Standard_Time(EC/개)']
    merged_df['to_be(Standard Time)'] = merged_df['Standard_Time(EC/개)']*(1-merged_df['자동화 1안'])
    merged_df['적정인력(명)'] = merged_df['to_be(Standard Time)'] / merged_df['TaktTime(SEC/개)']
    merged_df['도급비(백만원/월)'] = merged_df['월평균 도급비(백만원/월)'] / merged_df['현근무인원(인/日)'] * merged_df['적정인력(명)']



    ########  Step.15  ########

    # 컬럼들의 합계 계산
    sum_columns = ['월평균 도급비(백만원/월)', '현근무인원(인/日)',  'as_is(Standard Time)', 'to_be(Standard Time)', '적정인력(명)', '도급비(백만원/월)']
    column_sums = merged_df[sum_columns].sum()

    # 합계를 데이터프레임으로 변환
    sum_df1 = pd.DataFrame(column_sums, columns=['1안(4200백만원)'])
    ########  Step.16  ########
    merged_df['as_is'] = merged_df['Standard_Time(EC/개)']
    merged_df['to_be'] = merged_df['Standard_Time(EC/개)']*(1-merged_df['자동화 2안'])
    merged_df['적정인력(명)'] = merged_df['to_be'] / merged_df['TaktTime(SEC/개)']
    merged_df['도급비(백만원/월)'] = merged_df['월평균 도급비(백만원/월)'] / merged_df['현근무인원(인/日)'] * merged_df['적정인력(명)']


    ########  Step.17  ########
    # 컬럼들의 합계 계산
    sum_columns = ['월평균 도급비(백만원/월)', '현근무인원(인/日)',  'as_is(Standard Time)', 'to_be(Standard Time)', '적정인력(명)', '도급비(백만원/월)']
    column_sums = merged_df[sum_columns].sum()

    # 합계를 데이터프레임으로 변환
    sum_df2 = pd.DataFrame(column_sums, columns=['2안(8000백만원)'])
    ########  Step.18  ########
    # sum_df1과 sum_df2를 옆으로 붙이기
    combined_df = pd.concat([sum_df1, sum_df2], axis=1)
    combined_df=combined_df.transpose()
    asis[['total_적정인력', '도급비(백만원/월)']]

    combined_df['As-is_적정인력'] = asis['total_적정인력'].values[0]
    combined_df['As-is_월도급비'] = asis['도급비(백만원/월)'].values[0]
    combined_df['As-is_연간도급비'] = combined_df['As-is_월도급비'] *12
    combined_df['연간도급비(백만원/연)'] = combined_df['도급비(백만원/월)'] *12
    df_last= combined_df[[ '적정인력(명)', '연간도급비(백만원/연)', 'As-is_적정인력',
        'As-is_월도급비', 'As-is_연간도급비']].copy()

    df_last['도급인력 절감'] = df_last['As-is_적정인력'] - df_last['적정인력(명)']
    df_last['연간 도급비 절감(백만원/연)'] = df_last['As-is_연간도급비'] - df_last['연간도급비(백만원/연)']
    df_last['투자금액'] = [4200,8000]
    df_last['투자회수 기간(년차)'] = df_last['투자금액'] / df_last['연간 도급비 절감(백만원/연)']
    q1 = round(df_last,1)
    q1 = q1[['투자회수 기간(년차)']]
    q1['순작업시간비율'] = f'{순작업시간비율:.2f}'
    #q1.set_index('비중', inplace=True)
    q1 = q1.transpose()

    sum_df = pd.DataFrame(q1)
    results.append(sum_df)

# 결과 데이터프레임 생성
combined_df = pd.concat(results, axis=1)
combined_df.transpose()

<ipython-input-10-dde2834506d5>:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_last['도급인력 절감'] = df_last['As-is_적정인력'] - df_last['적정인력(명)']
<ipython-input-10-dde2834506d5>:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_last['연간 도급비 절감(백만원/연)'] = df_last['As-is_연간도급비'] - df_last['연간도급비(백만원/연)']
<ipython-input-10-dde2834506d5>:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

,투자회수 기간(년차),비중
1안(4200백만원),2.5,0.40
2안(8000백만원),3.0,0.40
1안(4200백만원),2.8,0.45
2안(8000백만원),3.4,0.45
1안(4200백만원),3.1,0.50
2안(8000백만원),3.8,0.50
1안(4200백만원),3.4,0.55
2안(8000백만원),4.2,0.55
1안(4200백만원),3.7,0.60
2안(8000백만원),4.5,0.60
